# PEFT - Prompt Tuning 예제

이 노트북은 PromptTuningConfig를 사용하여 LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct 모델을 NSMC 데이터셋으로 학습시키는 예제입니다.


In [1]:
# 필요한 라이브러리 설치
%pip install transformers==4.44.0
%pip install peft==0.11.1
%pip install datasets==2.20.0
%pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
%pip install accelerate==0.32.1



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.22.0.dev20250319+cu128
    Uninstalling torchvision-0.22.0.dev20250319+cu128:
      Successfully uninstalled torchvision-0.22.0.dev20250319+cu128
  Attempting uninstall: torchaudio
    Found ex

In [2]:
# 라이브러리 import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import PromptTuningConfig, get_peft_model, TaskType, PeftModel
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")


사용 중인 디바이스: cuda


In [3]:
# NSMC 데이터셋 로드
train_data = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_data = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

# 결측값 제거
train_data = train_data.dropna()
test_data = test_data.dropna()

# 학습 속도를 위해 샘플 수 제한
train_data = train_data.sample(n=5000, random_state=42).reset_index(drop=True)
test_data = test_data.sample(n=1000, random_state=42).reset_index(drop=True)

print(f"학습 데이터 크기: {len(train_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")
print(f"학습 데이터 예시:")
print(train_data.head())


학습 데이터 크기: 5000
테스트 데이터 크기: 1000
학습 데이터 예시:
        id                    document  label
0  7865795                      원본이 최고      1
1  5417631            스릴감과 훈훈함이 있는 영화.      1
2  8357466      굉장히 저평가되는 영화중 하나라고 생각함      1
3  8252946  정말영화같은이야기 영화여서 영화같은이야기가 좋다      1
4  7800452                 계기도없는데 이상하다      0


In [4]:
# 모델과 토크나이저 로드
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 패드 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"모델 로드 완료: {model_name}")
print(f"모델 파라미터 수: {model.num_parameters():,}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델 로드 완료: LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct
모델 파라미터 수: 2,405,327,360


In [5]:
# PromptTuningConfig 설정
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init="TEXT",
    num_virtual_tokens=10,
    prompt_tuning_init_text="다음 영화 리뷰를 분석하여 긍정인지 부정인지 판단하세요:",
    tokenizer_name_or_path=model_name
)

# PEFT 모델 생성
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

print("PromptTuning 설정 완료")


trainable params: 25,600 || all params: 2,405,352,960 || trainable%: 0.0011
PromptTuning 설정 완료


In [6]:
# 데이터 전처리 함수
def preprocess_data(examples):
    # 입력 텍스트 구성
    texts = []
    for document, label in zip(examples['document'], examples['label']):
        label_text = "긍정" if label == 1 else "부정"
        text = f"리뷰: {document}\n감정: {label_text}"
        texts.append(text)
    
    # 토크나이징
    model_inputs = tokenizer(
        texts,
        truncation=True,
        padding="max_length",  # padding=True 대신 명시적으로 max_length 사용
        max_length=256,
        return_tensors=None
    )
    
    # 라벨을 input_ids로 설정 (언어 모델링 태스크)
    model_inputs["labels"] = [ids.copy() for ids in model_inputs["input_ids"]]
    
    return model_inputs

# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# 전처리 적용
train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_data, batched=True, remove_columns=test_dataset.column_names)

print("데이터 전처리 완료")


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

데이터 전처리 완료


In [7]:
# 학습 인수 설정
training_args = TrainingArguments(
    output_dir="./prompt_tuning_output",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    logging_steps=50,
    save_steps=500,
    eval_strategy="steps",  # transformers 4.44.0에서 evaluation_strategy 대신 eval_strategy 사용
    eval_steps=500,
    learning_rate=1e-3,
    fp16=True,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    report_to=[]  # wandb 등 로깅 비활성화
)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

print("학습 설정 완료")


학습 설정 완료


In [8]:
# 모델 학습 시작
print("모델 학습을 시작합니다...")
trainer.train()

# 모델 저장
model.save_pretrained("./prompt_tuning_output")
tokenizer.save_pretrained("./prompt_tuning_output")

print("학습 완료 및 모델 저장")


모델 학습을 시작합니다...


Step,Training Loss,Validation Loss
500,0.334300,0.331833
1000,0.278400,0.329880
1500,0.342100,0.328427
2000,0.314200,0.327365
2500,0.299500,0.326485


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


학습 완료 및 모델 저장


In [11]:
# 학습된 모델로 추론 함수
def predict_sentiment(text):
    # 입력 텍스트 구성
    input_text = f"리뷰: {text}\n감정:"
    
    # 토크나이징
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    
    # GPU가 있는 경우 디바이스 이동
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # 추론
    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_ids = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        outputs = torch.cat([inputs["input_ids"], predicted_ids], dim=1)
            
    # 결과 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# 테스트 예시
test_reviews = [
    "이 영화 정말 재미있어요. 강력 추천합니다!",
    "최악의 영화였습니다. 시간 낭비예요.",
    "배우들의 연기가 훌륭했습니다.",
    "스토리가 너무 지루하고 예측 가능해요."
]

print("추론 결과 (generate 방식):")
for review in test_reviews:
    result = predict_sentiment(review)
    print(f"입력: {review}")
    print(f"결과: {result}")
    print("-" * 50)


추론 결과 (generate 방식):
입력: 이 영화 정말 재미있어요. 강력 추천합니다!
결과: 리뷰: 이 영화 정말 재미있어요. 강력 추천합니다!
감정: 긍정
--------------------------------------------------
입력: 최악의 영화였습니다. 시간 낭비예요.
결과: 리뷰: 최악의 영화였습니다. 시간 낭비예요.
감정: 부정
--------------------------------------------------
입력: 배우들의 연기가 훌륭했습니다.
결과: 리뷰: 배우들의 연기가 훌륭했습니다.
감정: 긍정
--------------------------------------------------
입력: 스토리가 너무 지루하고 예측 가능해요.
결과: 리뷰: 스토리가 너무 지루하고 예측 가능해요.
감정: 부정
--------------------------------------------------
